In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2024-03-29 13:49:05.483540: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 13:49:06.156778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 13:49:24.461359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/m.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

2024-03-29 13:50:14.051441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31133 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2024-03-29 13:50:14.052063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31133 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2024-03-29 13:50:14.052566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31133 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0
2024-03-29 13:50:14.053432: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31133 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

0
[-0.0002885490222741538, 0.0003282892920148939, -0.019098326023271106, -0.0008577121936363952, -0.009495578461804932, -0.012122703954155156, -8.356202308679173e-05, -0.00908370067809905, 0.008339124372620077, 0.0005373786517477441, -0.008213842418860473, 0.008399622906033244, -0.006626003821386561, -0.006077997051190387, -0.00045107251058329467, 0.004448384681973394, -0.001480572711943573, -0.011172260170139836, 0.019456682913049583, 0.014165191949770062, 0.01320617800661772, 0.0066018889645018675, 0.008782776224440347, 0.020050842234569544, 0.008136033473048698, 0.0006056491809527387, 0.002690954443812288, -0.003595316966520379, -0.0084017951126798, 0.003701719534582537, -0.011234020904840492, -0.014465247434138462, 0.009150716533574688, -0.004027364932220223, -0.0037519867086679426, -0.015852911681857762, -0.011054378630921846, 0.006332306035448099, 0.0009049823095637066, -0.0007399371910565833, 0.011293689204260993, 0.01597363933068685, 0.00600768303869792, 0.018763964781134845, 0

0.0

In [10]:
p

0.0

In [11]:
p

0.0

In [12]:
p

0.0

In [13]:
p

0.0